## **Cuaderno Datos Transporte 2022 - 2023**

In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
municipios_cluster = pd.read_csv("external_data\municipios_con_cluster.csv", sep=",")

### **estacionBus_100**

In [3]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"              # sep=";"
ESTACIONES_BUS_CSV = "data/transporte/paradas_bus.csv"         # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN ESTACIONES BUS (ya ancho)
# ============================================================
bus = pd.read_csv(ESTACIONES_BUS_CSV, sep=";", dtype=str)

bus["Nombre"] = bus["Nombre"].apply(normalize_nombre)
bus["key"] = bus["Nombre"].apply(canon_key)

year_cols = [c for c in bus.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    bus[c] = bus[c].apply(parse_es_number)

numero_estaciones_bus = bus[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs estaciones) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_bus = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_bus) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_estaciones = (
        numero_estaciones_bus[["key", "Nombre", col]]
        .rename(columns={col: "numero_estaciones_bus"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_estaciones, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["numero_estaciones_bus"] = pd.to_numeric(datos["numero_estaciones_bus"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Estaciones bus por 100 habitantes
    datos["estacionbus_100"] = ((datos["numero_estaciones_bus"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["estacionbus_100"].mean()
        desviacion = grupo["estacionbus_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "estacionbus_100"}
    )

    output_path = f"data_interfaz/transporte/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}estacionbus_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)

# Si SOLO quieres 2022 y 2023:
# procesar_anio(2022)
# procesar_anio(2023)


Años a procesar: [2023]
Archivo del 2023 exportado a data_interfaz/transporte/normalizacion_final/2023/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28636\251099874.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **Bus_100**

In [4]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
BUS_CSV = "data/transporte/total_buses.csv"                     # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN BUS (ya ancho)
# ============================================================
bus = pd.read_csv(BUS_CSV, sep=";", dtype=str)

bus["Nombre"] = bus["Nombre"].apply(normalize_nombre)
bus["key"] = bus["Nombre"].apply(canon_key)

year_cols = [c for c in bus.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    bus[c] = bus[c].apply(parse_es_number)

numero_bus_total = bus[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs bus) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_bus = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_bus) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_bus = (
        numero_bus_total[["key", "Nombre", col]]
        .rename(columns={col: "numero_bus_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_bus, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["numero_bus_total"] = pd.to_numeric(datos["numero_bus_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Bus por 100 habitantes
    datos["bus_100"] = ((datos["numero_bus_total"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["bus_100"].mean()
        desviacion = grupo["bus_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "bus_100"}
    )

    output_path = f"data_interfaz/transporte/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}bus_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)

# Si SOLO quieres 2022 y 2023:
# procesar_anio(2022)
# procesar_anio(2023)


Años a procesar: [2023]
Archivo del 2023 exportado a data_interfaz/transporte/normalizacion_final/2023/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28636\1171770252.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **EstacionTren_100**

In [5]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"              # sep=";"
ESTACIONES_TREN_CSV = "data/transporte/estaciones_tren.csv"       # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN ESTACIONES TREN (ya ancho)
# ============================================================
tren = pd.read_csv(ESTACIONES_TREN_CSV, sep=";", dtype=str)

tren["Nombre"] = tren["Nombre"].apply(normalize_nombre)
tren["key"] = tren["Nombre"].apply(canon_key)

year_cols = [c for c in tren.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    tren[c] = tren[c].apply(parse_es_number)

numero_estaciones_tren = tren[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs tren) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_tren = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_tren) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_estaciones = (
        numero_estaciones_tren[["key", "Nombre", col]]
        .rename(columns={col: "numero_estaciones_tren"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_estaciones, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["numero_estaciones_tren"] = pd.to_numeric(datos["numero_estaciones_tren"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Estaciones tren por 100 habitantes
    datos["estaciontren_100"] = ((datos["numero_estaciones_tren"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["estaciontren_100"].mean()
        desviacion = grupo["estaciontren_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "estaciontren_100"}
    )

    output_path = f"data_interfaz/transporte/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}estaciontren_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)

# Si SOLO quieres 2022 y 2023:
# procesar_anio(2022)
# procesar_anio(2023)


Años a procesar: [2023]
Archivo del 2023 exportado a data_interfaz/transporte/normalizacion_final/2023/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28636\218433434.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **distanciaCentro**

In [6]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
DISTANCIA_CSV = "data/transporte/distancia_al_centro.csv"       # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
#    (aunque no se use para el cálculo, lo dejamos por plantilla)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN DISTANCIA A CAPITAL (ya ancho)
# ============================================================
dist = pd.read_csv(DISTANCIA_CSV, sep=";", dtype=str)

dist["Nombre"] = dist["Nombre"].apply(normalize_nombre)
dist["key"] = dist["Nombre"].apply(canon_key)

year_cols = [c for c in dist.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    dist[c] = dist[c].apply(parse_es_number)

distancia_capital = dist[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs distancia)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_dist = {int(c) for c in year_cols}

years_to_process = sorted(years_pobl.intersection(years_dist))
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    distancia_capital_total = (
        distancia_capital[["key", "Nombre", col]]
        .rename(columns={col: "distancia_capital"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(distancia_capital_total, poblacion_anio, on="key", how="inner")

    datos["distancia_capital"] = pd.to_numeric(datos["distancia_capital"], errors="coerce")

    # Aquí la distancia ya es el indicador (no ratio)
    datos["distanciaCentro"] = datos["distancia_capital"].fillna(0)

    # Menor distancia = mejor
    datos["type"] = 0

    # Añadir cluster
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["distanciaCentro"].mean()
        desviacion = grupo["distanciaCentro"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "distanciaCentro"}
    )

    output_path = f"data_interfaz/transporte/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}distanciaCentro.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR (si solo quieres 2016, deja esto)
# ============================================================
procesar_anio(2024)

# Si quisieras todos:
# for y in years_to_process:
#     procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/transporte/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28636\2203937014.py:167: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **ServicioCoches_100**

In [7]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"                 # sep=";"
SERVICIO_COCHES_CSV = "data/transporte/total_turismos.csv"          # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN SERVICIO COCHES (ya ancho)
# ============================================================
sc = pd.read_csv(SERVICIO_COCHES_CSV, sep=";", dtype=str)

sc["Nombre"] = sc["Nombre"].apply(normalize_nombre)
sc["key"] = sc["Nombre"].apply(canon_key)

year_cols = [c for c in sc.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    sc[c] = sc[c].apply(parse_es_number)

numero_servicio_coches = sc[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs servicio coches) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_sc = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_sc) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_sc = (
        numero_servicio_coches[["key", "Nombre", col]]
        .rename(columns={col: "numero_servicio_coches"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_sc, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["numero_servicio_coches"] = pd.to_numeric(datos["numero_servicio_coches"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Servicio coches por 100 habitantes
    datos["servicioCoches_100"] = ((datos["numero_servicio_coches"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["servicioCoches_100"].mean()
        desviacion = grupo["servicioCoches_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "servicioCoches_100"}
    )

    output_path = f"data_interfaz/transporte/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}servicioCoches_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)

# Si SOLO quieres 2022 y 2023:
# procesar_anio(2022)
# procesar_anio(2023)


Años a procesar: [2023]
Archivo del 2023 exportado a data_interfaz/transporte/normalizacion_final/2023/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28636\3825818362.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
